# FINAL POYECT

## MODELAB PYTHON 2025

In [1]:
"""
Multivariate Environmental Analysis

Author: Ana Montoya
Python: 3.13.5
Libraries: numpy, pandas, matplotlib, scikit-learn, scipy

"""


'\nMultivariate Environmental Analysis\n\nAuthor: Ana Montoya\nPython: 3.13.5\nLibraries: numpy, pandas, matplotlib, scikit-learn, scipy\n\n'

In [2]:
# =========================
# Imports & Configuration
# =========================
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import MDS
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import braycurtis

In [4]:

# -------------------------
# Paths and global settings
# -------------------------
DATA_PATH = "./BD/bd_exercises_curse.csv"
OUT_DIR   = "outputs_mds"
FIG_DIR   = os.path.join(OUT_DIR, "01_figures")
os.makedirs(FIG_DIR, exist_ok=True)

RANDOM_STATE = 42

In [5]:
# =========================
#  Load the dataset
# =========================
def load_dataset(path) -> pd.DataFrame:
    df = pd.read_csv(path)
    return df
bd_data = load_dataset(DATA_PATH)

In [6]:
# =========================
#  Variable selection
# =========================
def select_variables(df: pd.DataFrame):
    """
    Choose numeric variables for analysis and group variables for interpretation.

    The function returns:
      numeric_cols: list of numeric columns present in the dataset
      group_cols:   list of grouping columns (if present)
    """
    # Candidate numeric features
    num_candidates = [
        "chlorophy_microg_l", "sal_psu", "temp_c", "depth_m",
        "do_mg_l", "do_percent_sat", "turbidity_fnu",
        "sp_cond_microsiemens_cm", "cond_microsiemens_cm",
        "dic_micromol_kg", "ta_micromol_kg"
    ]
    numeric_cols = [c for c in num_candidates if c in df.columns]

    # Optional metadata for plotting/interpretation
    group_candidates = ["estuary", "area", "season", "layer_depth", "station"]
    group_cols = [c for c in group_candidates if c in df.columns]

    # Save chosen columns
    pd.Series(numeric_cols).to_csv(os.path.join(OUT_DIR, "02_numeric_columns_used.csv"), index=False, header=["numeric_cols"])
    pd.Series(group_cols).to_csv(os.path.join(OUT_DIR, "02_group_columns_used.csv"), index=False, header=["group_cols"])
    return numeric_cols, group_cols

In [7]:
# =========================
#  Imputation + Standardization
# =========================
def impute_and_standardize(df: pd.DataFrame, numeric_cols):
    """
    Returns:
      X_imputed (np.array), X_scaled (np.array), z_df (DataFrame z-scores)
    """
    X_raw = df[numeric_cols].copy()
    imputer = SimpleImputer(strategy="median")
    X_imputed = imputer.fit_transform(X_raw)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_imputed)

    z_df = pd.DataFrame(X_scaled, columns=[f"z_{c}" for c in numeric_cols], index=df.index)
    z_df.to_csv(os.path.join(OUT_DIR, "03_zscores.csv"), index=False)
    return X_imputed, X_scaled, z_df


In [8]:
# =========================
#  Dissimilarity Matrices
# =========================
def compute_dissimilarities(X_scaled: np.ndarray, X_nonneg: np.ndarray):
    """
    Compute Euclidean (on standardized features) and Bray-Curtis (on non-negative features).
    Returns:
      D_euclid (n x n), D_bray (n x n)
    """
    # Euclidean on standardized vars
    D_euclid = pairwise_distances(X_scaled, metric="euclidean")

    # Bray-Curtis expects non-negative values
    n = X_nonneg.shape[0]
    D_bray = np.zeros((n, n), dtype=float)
    for i in range(n):
        for j in range(i + 1, n):
            d = braycurtis(X_nonneg[i], X_nonneg[j])
            D_bray[i, j] = d
            D_bray[j, i] = d

    # Save to CSV (square matrices)
    pd.DataFrame(D_euclid).to_csv(os.path.join(OUT_DIR, "04_D_euclidean.csv"), index=False)
    pd.DataFrame(D_bray).to_csv(os.path.join(OUT_DIR, "04_D_braycurtis.csv"), index=False)
    return D_euclid, D_bray
# =========================


In [9]:
# =========================
#  MDS (metric & non-metric)
# =========================
def run_mds(D: np.ndarray, metric: bool, random_state: int = RANDOM_STATE):
    """
    Run 2D MDS given a precomputed dissimilarity matrix.
    metric=True for metric MDS, False for non-metric MDS.
    Returns:
      Y (n x 2) configuration, stress (float)
    """
    mds = MDS(
        n_components=2,
        metric=metric,
        dissimilarity="precomputed",
        random_state=random_state,
        n_init=4,
        max_iter=500
        # Avoid using normalized_stress for broad compatibility across scikit-learn versions
    )
    Y = mds.fit_transform(D)
    stress = float(mds.stress_)
    return Y, stress

In [10]:
# =========================
#  Plots – single-purpose figures
# =========================
def scatter_mds(Y: np.ndarray, title: str, df: pd.DataFrame, group_cols):
    """
    Create a scatter of MDS coordinates.
    - Uses marker shape variation for one grouping column if available
    - No custom colors are set (follows the course constraint)
    """
    plt.figure(figsize=(6, 5))
    # Choose one grouping column for markers if available
    marker_by = group_cols[0] if len(group_cols) > 0 else None

    if marker_by and marker_by in df.columns:
        codes = pd.Categorical(df[marker_by]).codes
        marker_list = ["o", "s", "^", "D", "P", "X", "*", "v", "<", ">"]
        for mk in np.unique(codes):
            idx = np.where(codes == mk)[0]
            plt.scatter(Y[idx, 0], Y[idx, 1], marker=marker_list[mk % len(marker_list)])
    else:
        plt.scatter(Y[:, 0], Y[:, 1])

    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.title(title)
    plt.tight_layout()


def save_current_fig(filename: str):
    out_path = os.path.join(FIG_DIR, filename)
    plt.savefig(out_path, dpi=180)
    plt.close()
    return out_path


def simple_lonlat_map(df: pd.DataFrame, lon_col="longitude", lat_col="latitude", title="Sampling Locations (Lon/Lat)"):
    """
    Simple lon/lat scatter without basemap (keeps dependencies minimal).
    """
    if lon_col in df.columns and lat_col in df.columns:
        plt.figure(figsize=(6, 5))
        plt.scatter(df[lon_col], df[lat_col])
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.title(title)
        plt.tight_layout()
        return save_current_fig("00_map_lonlat.png")
    return None


In [11]:
# =========================
#  Markdown Report (equations & references)
# =========================
def write_markdown_report(numeric_cols, group_cols, stress_dict):
    """
    Write a simple Markdown report including equations and APA references.
    """
    numeric_cols_list = ", ".join(numeric_cols) if numeric_cols else "N/A"
    group_cols_list   = ", ".join(group_cols) if group_cols else "N/A"

    report = f"""
# Multidimensional Scaling (MDS) of Environmental Data – Step-by-Step

**Dataset:** {os.path.basename(DATA_PATH)}  
**Variables used (standardized):** {numeric_cols_list}  
**Group metadata (for interpretation):** {group_cols_list}

## 1. Preprocessing (Standardization)
We standardized each variable using z-scores:  
$$ z_{{i}} = \\frac{{x_{{i}} - \\mu}}{{\\sigma}} $$
where $x_i$ is the raw value, $\\mu$ is the feature mean, and $\\sigma$ is the feature standard deviation.

## 2. Dissimilarity Matrices
- **Euclidean** on standardized variables:  
$$ d_{{ij}}^{{(Euc)}} = \\sqrt{{\\sum_k (z_{{ik}} - z_{{jk}})^2 }} $$
- **Bray–Curtis** on non-negative variables:  
$$ d_{{ij}}^{{(BC)}} = \\frac{{\\sum_k |x_{{ik}} - x_{{jk}}|}}{{\\sum_k (x_{{ik}} + x_{{jk}})}} $$

## 3. Multidimensional Scaling (MDS)
We computed **metric** and **non-metric** MDS in 2D. The objective is to find a low-dimensional configuration $\\mathbf{{Y}}$ that preserves the original dissimilarities as distances.  
A commonly reported loss is Kruskal's stress:  
$$ \\mathrm{{Stress}} = \\sqrt{{ \\frac{{\\sum_{{i<j}} (d_{{ij}} - \\hat{{d}}_{{ij}})^2 }}{{\\sum_{{i<j}} d_{{ij}}^2 }} }} $$
where $d_{{ij}}$ are observed dissimilarities and $\\hat{{d}}_{{ij}}$ are distances in the MDS map.

### Results (Stress)
- Metric MDS (Euclidean): **{stress_dict.get('metric_euclid', 'n/a'):.6f}**
- Non-Metric MDS (Euclidean): **{stress_dict.get('nonmetric_euclid', 'n/a'):.6f}**
- Metric MDS (Bray–Curtis): **{stress_dict.get('metric_bray', 'n/a'):.6f}**

Lower stress indicates better preservation of the original dissimilarities in 2D.

## 4. Interpretation Guide
- **Nearby points** in the MDS plots indicate **similar environmental conditions**.
- **Far apart points** indicate **contrasting conditions** (e.g., plume vs. coastal sites).
- Add metadata (e.g., *estuary*, *area*, *layer_depth*) to better interpret potential ecological gradients.

## 5. References (APA-style)
- Borg, I., Groenen, P. J. F., & Mair, P. (2013). *Applied Multidimensional Scaling*. Springer.
- Gower, J. C. (1966). Some distance properties of latent root and vector methods used in multivariate analysis. *Biometrika, 53*(3-4), 325–338.
- Kruskal, J. B. (1964). Nonmetric multidimensional scaling: A numerical method. *Psychometrika, 29*(2), 115–129.
- Legendre, P., & Legendre, L. (2012). *Numerical Ecology* (3rd ed.). Elsevier.
- Mardia, K. V., Kent, J. T., & Bibby, J. M. (1979). *Multivariate Analysis*. Academic Press.

---
"""
    with open(os.path.join(OUT_DIR, "99_REPORT.md"), "w", encoding="utf-8") as f:
        f.write(report)

In [16]:
# =========================
#  Main: run the pipeline
# =========================
def main():
    #  Make directories
    os.makedirs(OUT_DIR, exist_ok=True)
    os.makedirs(FIG_DIR, exist_ok=True)

    # Load data
    df = load_dataset(DATA_PATH)

    # Variable selection
    map_path = simple_lonlat_map(df, lon_col="longitude", lat_col="latitude")
    numeric_cols, group_cols = select_variables(df)

    # Impute + Standardize (z-scores)
    X_imputed, X_scaled, z_df = impute_and_standardize(df, numeric_cols)

    # For Bray-Curtis, ensure non-negative inputs
    X_nonneg = np.clip(X_imputed, a_min=0, a_max=None)

    # Dissimilarity matrices
    D_euclid, D_bray = compute_dissimilarities(X_scaled, X_nonneg)

    # MDS
    # Metric MDS (Euclidean)
    Y_metric_euclid, stress_metric_euclid = run_mds(D_euclid, metric=True)

    # Non-Metric MDS (Euclidean)
    Y_nonmetric_euclid, stress_nonmetric_euclid = run_mds(D_euclid, metric=False)

    # Metric MDS (Bray–Curtis)
    Y_metric_bray, stress_metric_bray = run_mds(D_bray, metric=True)

    # Save coordinates with metadata to CSV
    out_coords = pd.DataFrame({
        "mds_metric_euclid_dim1": Y_metric_euclid[:, 0],
        "mds_metric_euclid_dim2": Y_metric_euclid[:, 1],
        "mds_nonmetric_euclid_dim1": Y_nonmetric_euclid[:, 0],
        "mds_nonmetric_euclid_dim2": Y_nonmetric_euclid[:, 1],
        "mds_metric_bray_dim1": Y_metric_bray[:, 0],
        "mds_metric_bray_dim2": Y_metric_bray[:, 1],
    })
    # Append group columns if available
    for c in group_cols:
        out_coords[c] = df[c].values

    out_coords.to_csv(os.path.join(OUT_DIR, "05_mds_coordinates.csv"), index=False)

    # Plots (one chart per file)
    # Metric MDS (Euclidean)
    scatter_mds(Y_metric_euclid, f"Metric MDS (Euclidean)\nStress={stress_metric_euclid:.4f}", df, group_cols)
    save_current_fig("06_metric_mds_euclidean.png")

    # Non-Metric MDS (Euclidean)
    scatter_mds(Y_nonmetric_euclid, f"Non-Metric MDS (Euclidean)\nStress={stress_nonmetric_euclid:.4f}", df, group_cols)
    save_current_fig("07_nonmetric_mds_euclidean.png")

    # Metric MDS (Bray-Curtis)
    scatter_mds(Y_metric_bray, f"Metric MDS (Bray-Curtis)\nStress={stress_metric_bray:.4f}", df, group_cols)
    save_current_fig("08_metric_mds_braycurtis.png")

    # Write Markdown report with equations and references
    stress_dict = {
        "metric_euclid": stress_metric_euclid,
        "nonmetric_euclid": stress_nonmetric_euclid,
        "metric_bray": stress_metric_bray
    }
    write_markdown_report(numeric_cols, group_cols, stress_dict)

    print("\nDone:")
    print("- 01_figures")
    print("- 02_numeric_columns_used.csv / 02_group_columns_used.csv")
    print("- 03_zscores.csv")
    print("- 04_D_euclidean.csv / 04_D_braycurtis.csv")
    print("- 05_mds_coordinates.csv")
    print("- 06_metric_mds_euclidean.png / 07_nonmetric_mds_euclidean.png / 08_metric_mds_braycurtis.png")
    if map_path:
        print("- 00_map_lonlat.png (simple lon/lat map)")
    print("- 90_combined_original_z_mds.csv")
    print("- 99_REPORT.md")
    




In [17]:
# Execute this line to run the final project
main()


Done:
- 01_figures
- 02_numeric_columns_used.csv / 02_group_columns_used.csv
- 03_zscores.csv
- 04_D_euclidean.csv / 04_D_braycurtis.csv
- 05_mds_coordinates.csv
- 06_metric_mds_euclidean.png / 07_nonmetric_mds_euclidean.png / 08_metric_mds_braycurtis.png
- 00_map_lonlat.png (simple lon/lat map)
- 90_combined_original_z_mds.csv
- 99_REPORT.md
